In [1]:
%load_ext autoreload
%autoreload 2

import sys
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns

sys.path.insert(0, "..") 
from src.data import make_dataset
from src.features import build_features
from src.visualization import visualize

pd.set_option("display.precision", 5)
pd.set_option("display.max.columns", None)
pd.options.display.float_format = '{:6.5f}'.format



# Naive model

Het eerste model wat we willen proberen is de 'naive voorspelling'. Dit is een base line model ter beoordeling van de resultaten van de overige modellen.

In [2]:
# Shape of the Bond data is simple: Bond, rate_dt, price
df_price = make_dataset.read_pkl('price')

# Create a model that will predict 10 steps into the future, given as much data as available.


2022-01-03 16:00:04.268 | INFO     | src.data.make_dataset:read_pkl:358 - Load preprocessed price data
